# Моделирование Теоремы Гливенко-Контелли

Мы смоделируем не саму теорему Гливенко-Контелли, а более слабое утверждение:
$$
\sup\limits_{x} |F(x)-\overline{F_n}(x)| \xrightarrow{P} 0 \Longleftrightarrow P(\sup\limits_{x} |F(x)-\overline{F_n}(x)| \geq \varepsilon) \longrightarrow 0, \text{ при } n \to \infty
$$
Используем распределение Парето для моделирования этого утверждения. Напомним, что функция распределения Парето выглядит следующим образом:
$$
F(x) = \begin{cases}
1 - \frac{1}{x^\alpha}, & x \geq 1 \cr
0, & x < 1
\end{cases}
$$
Это то самое распределение, которое гласит, что 20% людей владеют 80% богатства. Запишем также его плотность:
$$
p(x) = \begin{cases}
 \frac{\alpha}{x^{\alpha+1}}, & x \geq 1 \cr
0, & x < 1
\end{cases}
$$

In [47]:
# Импортируем необходимые библиотки
import numpy as np
import scipy
import matplotlib.pyplot as plt

Реализуем функцию распределения Парето:

In [70]:
def Pareto_distribution_function(x, alpha):
    '''
    Функция, вычисляющая значение функции распределения Парето с параметром alpha в точке x (x >= 1)
    '''
    return 1 - 1 / (x ** alpha)

Зададим параметры моделирования:

In [73]:
alpha = 4 # Параметр распределения Парето
n = 30000 # максимальное количество элементов выборки
iterations = 100 # количество итераций для каждой выборки определенного размера (для подсчета вероятности)
epsilon = 0.01 
interval = np.linspace(1, (10**(4/alpha)), 10000) # отрезок, который охватывает значения, состовляющие вероятность 0,9999 
F = Pareto_distribution_function(interval, alpha) # Функция распределение Парето для точек из interval

Само моделирование происходит следующим образом: \
Во-первых, итерация по размеру выборки (от 1 до n). Тут все понятно: параметр ni подается в функцию scipy.stats.pareto.rvs и на выходе мы получаем ni точек сгенерированный по распределению Парето с параметром alpha. \
Второй цикл нам нужен для вычисления вероятности: мы генерируем определенное количество ($iterations$) выборок одной и той же длины. Для каждой из этих выборок считаем максимальную разницу между теоретической и эмпирической функциями распределения ($supremum$). Затем считаем количество максимальных разниц, значения которых больше $epsilon$ ($n_true$). И далее получаем вероятность для выборки данной длины: $P(|F(x)-\overline{F_n}(x)| \geq \varepsilon) = \frac{n\_true}{iterations}$. \
В самом конце строим график для визуализации.

In [ ]:
P = [] # Список для хранения вероятностей для выборок длины от 1 до n
for ni in range(1, n + 1):
    supremum = [] # Список для хранения максимумов отклонений генеральной и эмпирической функций распределения 
                  # для выборок определенной длины
    for mi in range(iterations):
        data = scipy.stats.pareto.rvs(alpha, size=ni)   # ni точек сгенерированный по распределению Парето
        F_n = scipy.stats.ecdf(data)                    # Эмпирическая функция распределения
        diff = np.abs(F - F_n.cdf.evaluate(interval))   # Разница между функциями распределения
        supremum.append(np.max(diff))                   
    supremum =  np.array(supremum)
    n_true = len(supremum[supremum >= epsilon])         # Количество выборок одной длины, которые >= epsilon
    P.append( n_true / iterations)
plt.plot(P)
plt.xlabel('n')
plt.ylabel('P')
plt.minorticks_on()
plt.grid(which='major', color = '#444', linewidth = 1)
plt.grid(which='minor', color='#aaa', ls=':')
plt.show()